# Offline Augmentation

한 번 파일이 생성되면 지우기 번거로울 수 있으므로, 데이터 복사 후 적용하길 권장합니다.

In [10]:
import os
import glob
import numpy as np
from tqdm import tqdm
from PIL import Image
import albumentations as A
import re

In [11]:
def _parse(p):
    """
    path를 파싱해 라벨 리턴
    """
    p = str(p)
    match = re.search('_(.+)_Asian_(\d+)/(.*)[\.-]', p)
    if match and len(match.groups()) == 3:
        gender, age, mask = match.groups()
        gender = 0 if gender == 'male' else 1
        age = int(age)
        if age < 30:
            age = 0
        elif age < 60:
            age = 1
        else:
            age = 2

        if mask.startswith('normal'):
            mask = 2
        elif mask.startswith('incorrect'):
            mask = 1
        else:
            mask = 0
        return mask, gender, age
    else:
        raise Exception(f'Cannot parse label from the path: {p}')

def is_image(path):
    exts = ['jpg', 'jpeg', 'png']
    for ext in exts:
        if path.endswith(ext):
            return True
    return False

def glob_image_paths(root):
    """
    root 하위에 있는 이미지 경로들을 리턴하는 함수
    Args:
        root: 이미지가 있는 경로
    """
    paths = glob.glob(os.path.join(root, '**/*'), recursive=True)
    paths = [f for f in paths if is_image(f)]
    return paths

def offline_aug(paths, suffix, transform):
    """
    OfflineAug를 위한 함수.
    paths로 주어진 이미지 파일들에 transform을 적용한 이미지를 새로 생성함.
    새로 생성된 이미지는 같은 디렉터리에 저장되며,
    '{원래파일명}-{suffix}.jpg' 형식의 파일명으로 저장됨.
    Args:
        paths: OfflineAug를 적용할 이미지 경로들
        suffix: 파일명에 사용할 접미사.
        transform: 이미지에 적용할 변환. Albumentation 권장
    """
    for path in tqdm(paths):
        newpath = path[0] + path[1:].replace('.', f'-{suffix}.')

        im = np.array(Image.open(path))
        im = transform(image=im)['image']
        im = Image.fromarray(im)
        im.save(newpath)

    print('Offline Augmentation 완료')

In [12]:
ROOT = '/opt/ml/input/60HFRotate45' # ex) /opt/ml/input/data/train/images
paths = glob_image_paths(ROOT)
paths = [p for p in paths if _parse(p)[2] == 2]

print('Offline Aug 전 이미지 개수:', len(paths))

Offline Aug 전 이미지 개수: 1344


In [13]:
# 원하는 Transform 정의
t = A.Compose([
    A.HorizontalFlip(p=1.0)
])

L = offline_aug(paths, 'HFlip', t)

100%|██████████| 1344/1344 [00:14<00:00, 92.64it/s]

Offline Augmentation 완료


In [14]:
t = A.Compose([
    A.Rotate(45, p=1, always_apply=True)
])
L = offline_aug(paths, 'R45', t)

100%|██████████| 1344/1344 [00:17<00:00, 76.27it/s]

Offline Augmentation 완료


In [15]:
paths = glob_image_paths(ROOT)
print('Offline Aug 후 이미지 개수:', len(paths))

Offline Aug 후 이미지 개수: 21588
